# 2.3. Automatic Differentiation
- loss function을 증가시키거나, 감소시키는 영향력을 파악함으로써 파라미터를 업데이트 
- 미분을 통해 이것이 가능하며, 이를 자동화시키는 것이 '미분 자동화'

### 2.3.1. A Simple Example
$y = 2\mathbf{x}^{\top}\mathbf{x}$ 의 미분

In [1]:
from mxnet import autograd,nd

In [2]:
x=nd.arange(4).reshape((4,1))
print(x)


[[0.]
 [1.]
 [2.]
 [3.]]
<NDArray 4x1 @cpu(0)>


x를 미분할 변수로 지정

In [3]:
x.attach_grad()

미분 대상인 식 할당. 계산 그래프를 내부에 구성함. 각 노드마다의 미분값을 기록. 

In [4]:
with autograd.record():
    y=2*nd.dot(x.T,x)
print(y)


[[28.]]
<NDArray 1x1 @cpu(0)>


오류를 역전파(Backward) 형태로 반영

In [5]:
y.backward()

반영된 parameter 미분값

In [6]:
x.grad


[[ 0.]
 [ 4.]
 [ 8.]
 [12.]]
<NDArray 4x1 @cpu(0)>

우리가 알고 있는 미분값과 같음

In [10]:
print((x.grad-4*x).norm().asscalar()==0)

True


### 2.3.2. Training Mode and Prediction Mode
미분이 학습에서만 일어난다고 가정. 

In [12]:
print(autograd.is_training())
with autograd.record():
    print(autograd.is_training())

False
True


### 2.3.3. Computing the Gradient of Python Control Flow
control flow : 선택이 내려질 수 있도록 조건과 결과들을 순서화한 것
단일 식이 아니라 control flow에 있는 변수라도 gradient를 구할 수 있음

In [13]:
def f(a):
    b = a * 2
    while b.norm().asscalar() < 1000:
        b = b * 2
    if b.sum().asscalar() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [14]:
a = nd.random.normal(shape=1)
a.attach_grad()
with autograd.record():
    d = f(a)
d.backward()

동일한 방식으로 수행시 수치계산과 동일한 값이 나옴. 선형식이니까 a로 나누면 됨

In [18]:
print((a.grad - (d / a)).norm().asscalar()==0)

True


### 2.3.4. Head gradients and the chain rule
chain rule을 이용하여 아래 식을 계산 
$$\frac{d}{dx} z(y(x)) = \frac{dz(y)}{dy} \frac{dy(x)}{dx}.$$ 
$\frac{dz}{dy}$ 를 head gradient로 해서 그 값을 $\frac{dy}{dx}$ 에 전달함


In [19]:
with autograd.record():
    y=x*2
y.attach_grad()
with autograd.record():
    z=y*x
z.backward()
y.backward(y.grad)
x.grad==2*x


[[1.]
 [1.]
 [1.]
 [1.]]
<NDArray 4x1 @cpu(0)>